In [13]:
import pymysql
import random

In [14]:
Category_Choices = [
    'Toners', 
    'Moisturizers', 
    'Sunscreen', 
    'Serum, Essence&Ampoules', 
    'Acne&Blemish Treatments', 
    'Exfoliating Scrubs&Peeling Gel', 
    'Eye Cream'
]

Ingredients_Choices = [
    "UV Protection",
    "Brightening",
    "Acne-Fighting",
    "Promotes Wound Healing",
    "Anti-Aging"
]

In [25]:
conn = pymysql.connect( host="127.0.0.1", port=3306, db="cosmetic_product",
                        user="humanda", passwd="humanda")
                               
cursor = conn.cursor()

    
def recommend_products(category, ingredients, min_count):
    if not ingredients:
        return None

    placeholders = ', '.join(['%s'] * len(ingredients))

    query = f"""
    SELECT p.product_no, p.category, p.product_name, p.url
    FROM product p
    JOIN ingredient i ON p.product_no = i.product_no
    WHERE p.category = %s 
    AND i.ingredient IN ({placeholders}) 
    AND i.count >= %s
    GROUP BY p.product_no
    HAVING COUNT(DISTINCT i.ingredient) = %s;
    """

    params = [category] + ingredients + [min_count] + [len(ingredients)]
    cursor.execute(query, params)

    products = cursor.fetchall()

    # 제품 데이터가 없을 경우
    if not products:
        return None

    # 제품 리스트
    product_list = [
        {"product_no": pid, "category": cat, "product_name": name, "url": url} 
        for pid, cat, name, url in products
    ]

    recommended = random.sample(product_list, min(5, len(product_list)))

    return recommended

def user_input_simulation():
    # 카테고리 선택
    print("\n📌 선택 가능한 카테고리:")
    for index, cat in enumerate(Category_Choices, start=1):
        print(f"{index}. {cat}")

    category_input = int(input("\n원하는 카테고리 번호를 선택하세요: ")) - 1
    if 0 <= category_input < len(Category_Choices):
        category = Category_Choices[category_input]
    else:
        print("❌ 잘못된 선택입니다.")
        return
    
    # 성분 선택
    print("\n📌 선택 가능한 성분 (쉼표로 구분하여 입력하세요, 최소 1개 이상 선택 가능): ")
    for index, ing in enumerate(Ingredients_Choices, start=1):
        print(f"{index}. {ing}")

    ingredient_input = input("\n찾고 싶은 성분의 번호를 입력하세요: ")
    ingredient_indices = [int(i.strip()) - 1 for i in ingredient_input.split(",") if i.strip().isdigit()]

    if any(idx < 0 or idx >= len(Ingredients_Choices) for idx in ingredient_indices):
        print("❌ 잘못된 성분 선택입니다.")
        return

    selected_ingredients = [Ingredients_Choices[i] for i in ingredient_indices]

    # count >= 3 검색
    recommended_products = recommend_products(category, selected_ingredients, 3)

    # count >= 2 검색
    if not recommended_products:
        recommended_products = recommend_products(category, selected_ingredients, 2)

    # count >= 1 검색
    if not recommended_products:
        recommended_products = recommend_products(category, selected_ingredients, 1)

    # 해당사항 없음
    if recommended_products:
        print("\n🔹 추천 제품 리스트:")
        for index, product in enumerate(recommended_products, start=1):
            print(f"{index}. {product['product_name']} ({product['url']})")
    else:
        print("\n❌ 해당 조건에 맞는 제품이 없습니다.")

# 실행
user_input_simulation()

# 연결 종료
cursor.close()
conn.close()


📌 선택 가능한 카테고리:
1. Toners
2. Moisturizers
3. Sunscreen
4. Serum, Essence&Ampoules
5. Acne&Blemish Treatments
6. Exfoliating Scrubs&Peeling Gel
7. Eye Cream

📌 선택 가능한 성분 (쉼표로 구분하여 입력하세요, 최소 1개 이상 선택 가능): 
1. UV Protection
2. Brightening
3. Acne-Fighting
4. Promotes Wound Healing
5. Anti-Aging

🔹 추천 제품 리스트:
1. AC Clear Moisture Pure N Cream (https://www.skincarisma.com/products/the-plant-base/ac-clear-moisture-pure-n-cream)
2. Oil Control Lotion (https://www.skincarisma.com/products/dermalogica/oil-control-lotion)
3. Concentrated Ginseng Renewing Cream EX (https://www.skincarisma.com/products/sulwhasoo/concentrated-ginseng-renewing-cream-ex)
4. Age Intervention Enlighten Plus (https://www.skincarisma.com/products/jan-marini-skin-research/age-intervention-enlighten-plus)
5. Medicated Whitening Gel (https://www.skincarisma.com/products/perfect-one/medicated-whitening-gel)
